In [0]:
import numpy as np
import pickle
import pandas as pd
import re
import string
import glob

In [0]:
path = glob.iglob('./websitesblog/*.csv')

In [0]:
list_ = []

for file_ in path:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)

frame = pd.concat(list_, axis = 0,)
#data = pd.read_csv('./websitesblog/website_123weddingcards.csv')

In [0]:
frame['text'].head()

0    Your Account,Access account and orders,\r\n\t\...
1    Your Account,Access account and orders,\r\n\t\...
2    Your Account,Access account and orders,Shop Cu...
3    Your Account,Access account and orders,Shop Cu...
4    Your Account,Access account and orders,SUBMIT,...
Name: text, dtype: object

In [0]:
lists_data = [re.sub('\s{2,}', ' ', re.sub(r'[^\x00-\x7f]','',re.sub('[\t\r\n,)([\]\d!%|!#\$%\&\*\+,.\-/:;\<=\>\?@\^_`{\|}~]',' ',str(i)))).lower().replace(r"'",'').replace('"','') for i in frame['text']]

In [0]:
for i in range(len(lists_data)):
    lists_data[i] = lists_data[i][:-1]

In [0]:
len(lists_data)

1866

In [0]:
with open('spelling_data_pruned.obj', 'rb') as f:
    training_data = pickle.load(f)

In [0]:
train_texts = [re.sub(r'[^\x00-\x7f]','',re.sub('[\t\r\n,)([\]\d!%|!#$%&*+,.-/:;<=>?@^_`{|}~?]','',str(i))).strip().lower() for i in training_data]

In [0]:
lists_data.extend(train_texts)
lists_data[-5:-1]

['im interested in christian wedding family cards',
 'show me filters with price range',
 'i want wedding cards price in range of  per card',
 'i looking for cards']

In [0]:
train_texts_split = [i.split() for i in lists_data]
training_data = []
for i in train_texts_split:
    for  j in i:
        training_data.append(j)
for k,i in enumerate(training_data):
    if len(i) == 1:
        del training_data[k]
len(training_data)

543904

[4, 2, 4, 5]


In [0]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z','1','2','3','4','5','6','7','8','9','0','!','@','#','$','%'
          '^','&','*','(',')','<','>',',','/','\\','|',':',';','~','`','+','-','.']

def noise_maker(sentence, threshold):
    '''Relocate, remove, or add characters to create spelling mistakes'''
    
    noisy_sentence = []
    i = 0
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        # Most characters will be correct since the threshold value is high
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            # ~33% chance characters will swap locations
            if new_random > 0.67:
                if i == (len(sentence) - 1):
                    # If last character in sentence, it will not be typed
                    continue
                else:
                    # if any other character, swap order with following character
                    noisy_sentence.append(sentence[i+1])
                    noisy_sentence.append(sentence[i])
                    i += 1
            # ~33% chance an extra lower case letter will be added to the sentence
            elif new_random < 0.33:
                random_letter = np.random.choice(letters, 1)[0]
                noisy_sentence.append(random_letter)
                noisy_sentence.append(sentence[i])
            # ~33% chance a character will not be typed
            else:
                pass     
        i += 1
    return noisy_sentence

In [0]:
# Check to ensure noise_maker is making mistakes correctly.
threshold = 0.5
for sentence in ['hindu']:
    print(sentence)
    d = noise_maker(sentence, threshold)
    print()
print(''.join(map(str,d)))

hindu

hnud


In [0]:
training_input = training_data.copy()
training_output = training_data.copy()

In [0]:
for k in [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]:
    for i in range(20):
        for m,sentence in enumerate(training_data):
            threshold = k
            training_input.append(''.join(map(str,noise_maker(sentence, threshold))))
            training_output.append(sentence)

KeyboardInterrupt: 

In [0]:
print(len(training_input),len(training_output))

44503658 44503658


In [0]:
with open('./new_dataset/spell_sentence/spelling_check_one_word_data_pruned_new_dataset','wb') as f:
    pickle.dump([training_input,training_output],f)